# Measurements Kata Workbook, Part 2

The [Measurements Kata Workbook, Part 1]() is still work in progress and will include the solutions of the tasks from Part I. This workbook continues the explanations for the tasks in Part II.

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20082513

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


In [ ]:
%workspace reload

## Part II*. Discriminating Nonorthogonal States

### <a name="task-21"></a>Task 2.1*. $|0\rangle$ or $|+\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|+\rangle$ state.

**Output:**  `true` if the qubit was in the $|0\rangle$ state, or `false` if it was in the $|+\rangle$ state. The state of the qubit at the end of the operation does not matter.

In this task your solution will be called multiple times, with one of the states picked with equal probability every time. You have to get overall accuracy of at least 80%.

> This task is an example of quantum hypothesis testing, or state discrimination with minimum error.

### Solution

Let ${|E_a\rangle, |E_b\rangle}$ be a measurement with two outcomes $a$ and $b$, which we identify with the answers, i.e., outcome "a" means we answer "state was |0\rangle" and outcome "b" means we answer "state was |+\rangle". Then we define

* $P(a|0)$ = probability to observe first outcome given that the state was $|0\rangle$
* $P(b|0)$ = probability to observe second outcome given that the state was $|0\rangle$
* $P(a|+)$ = probability to observe first outcome given that the state was $|+\rangle$
* $P(b|+)$ = probability to observe second outcome given that the state was $|+\rangle$

The task is to maximize the probability to be correct on a single shot experiment, which is the same as to minimize the probability to be wrong on a single shot. 

Since the task promises uniform prior distribution of the inputs $|0\rangle$ and $|+\rangle$, i.e., $P(+) = P(0) = \frac{1}{2}$, we get the following expression for the probability of giving a correct answer:

$$P_{correct} = P(0) P(a|0) + P(+) P(b|+) = \frac{1}{2} (P(a|0) + P(b|+))$$

We can represent our measurement as a von Neumann measurement of the following form:

$$|E_a\rangle = R_y(2\alpha) \begin{bmatrix} 1\\0\end{bmatrix} = \begin{bmatrix}\cos(\alpha) \\ \sin(\alpha)\end{bmatrix}, |E_b\rangle = R_y(2\alpha) \begin{bmatrix} 0\\1\end{bmatrix} = \begin{bmatrix}-\sin(\alpha) \\ \cos(\alpha)\end{bmatrix}$$

<img src="./img/Task201_Rotation.png" width="70%"/>

Using this representation, we can express our probabilities as follows:

$$P(a|0) = |\langle E_a|0\rangle|^2 = \cos^2(\alpha)$$

$$P(b|+) = |\langle E_b|+\rangle|^2 = \frac{1}{2} - \cos(\alpha) \sin(\alpha)$$

$$P_{correct} = \frac{1}{2} (\frac{1}{2} + \cos^2(\alpha) - \cos(\alpha) \sin(\alpha))$$

Maximizing this for $\alpha$, we get max $P_{success} = \frac{1}{2} (1 + \frac{1}{\sqrt{2}}) = 0.8535...$, which is attained for $\alpha = -\pi/8$.

This means that $|E_a\rangle$ and $|E_b\rangle$ are the result of rotating $|0\rangle$ and $|1\rangle$, respectively, by $-\pi/8$. If we rotate the whole system by $-\alpha = \pi/8$, we will get $|E_a\rangle=|0\rangle$ and $|E_b\rangle=|1\rangle$, and a measurement in the computational basis will give us the correct result with a probability of 85%.

> In Q#, rotating the input state by some angle $\theta$ can be done by applying $Ry$ gate with angle parameter $2\theta$.

In [ ]:
%kata T201_IsQubitZeroOrPlus_Test

open Microsoft.Quantum.Math;

operation IsQubitPlusOrZero (q : Qubit) : Bool {
    Ry(0.25 * PI(), q);
    return M(q) == Zero;
}

[Return to task 2.1 of the Measurements kata.](./Measurements.ipynb#task-21)

### <a name="task-22"></a>Task 2.2**. $|0\rangle$, $|+\rangle$ or inconclusive?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|+\rangle$ state.

**Output:** 

* 0 if the qubit was in the $|0\rangle$ state, 
* 1 if it was in the $|+\rangle$ state,
* -1 if you can't decide, i.e., an "inconclusive" result. 

Your solution:

* should never give 0 or 1 answer incorrectly (i.e., identify $|0\rangle$ as 1 or $|+\rangle$ as 0),
* will be called multiple times, with one of the states picked with equal probability every time,
* may give an inconclusive (-1) answer in at most 80% of all the cases,
* must correctly identify the $|0\rangle$ state as 0 in at least 10% of all the cases,
* must correctly identify the $|1\rangle$ state as 1 in at least 10% of all the cases.

The state of the qubit at the end of the operation does not matter.

> This task is an example of unambiguous state discrimination.


### Solution

A simple strategy that gives an inconclusive result with probability 0.75 and never errs in case it yields a conclusive result can be obtained from randomizing the choice of measurement basis between the computational basis (std) and the Hadamard basis (had).

Notice that when measured in the standard basis, the state $|0\rangle$ will always lead to the outcome "0", and the state $|+\rangle$ will lead to outcomes "0" and "1" with probability $\frac12$ each. This means that if we measure "1", we can with certainty conclude that the state was $|+\rangle$.

A similar argument applies to the scenario where we measure in the Hadamard basis, where $|0\rangle$ can lead to both "+" and "-" outcomes, and $|+\rangle$ always leads to "+". Then if we measured "-", we can with certainty conclude that the state was $|0\rangle$.

This leads to the following scenarios (shown are the conditional probabilities
of the resulting answers in each of the above scenarios).


   State     | Basis |    P(0)   |    P(1)   |   P(-1)
-------------|-------|-----------|-----------|----------
 $|0\rangle$ |  std  |     $0$   |     $0$   |    $1$
 $|+\rangle$ |  std  |     $0$   | $\frac12$ | $\frac12$
 $|0\rangle$ |  had  | $\frac12$ |     $0$   | $\frac12$
 $|+\rangle$ |  had  |     $0$   |     $0$   |    $1$

> The easiest way to implement the measurement in the Hadamard basis in Q# is to apply an H gate followed by a regular measurement in computational basis.

In [ ]:
%kata T202_IsQubitZeroOrPlusSimpleUSD_Test

open Microsoft.Quantum.Random;

operation IsQubitPlusZeroOrInconclusiveSimpleUSD (q : Qubit) : Int {
    let basis = DrawRandomInt(0, 1);

    // randomize over std and had measurement bases
    if (basis == 0) {
        // use standard basis
        let result = M(q);
        // result can be One only if the state was |+⟩; if result is Zero we can't be certain
        return result == One ? 1 | -1;
    }
    else {
        // use Hadamard basis
        H(q);
        let result = M(q);
        // result can be One only if the state was |0⟩; if result is Zero we can't be certain
        return result == One ? 0 | -1;
    }
}

[Return to task 2.2 of the Measurements kata.](./Measurements.ipynb#task-22)

### <a name="peres-wooters-game"></a>Task 2.3**.  Peres/Wooters game

**Input:** A qubit which is guaranteed to be in one of the three states:

* $|A\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + |1\rangle \big)$,
* $|B\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + \omega |1\rangle \big)$,
* $|C\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + \omega^2 |1\rangle \big)$,

Here $\omega = e^{2i \pi/ 3}$.

**Output:** 

* 1 or 2 if the qubit was in the $|A\rangle$ state, 
* 0 or 2 if the qubit was in the $|B\rangle$ state, 
* 0 or 1 if the qubit was in the $|C\rangle$ state.

You are never allowed to give an incorrect answer. Your solution will be called multiple times, with one of the states picked with equal probability every time.

The state of the qubit at the end of the operation does not matter. 

### Solution

> The task is a simple game inspired by a quantum detection problem due to Holevo<sup><a href="#peres-wooters-game-ref1">[1]</a></sup> and Peres/Wootters<sup><a href="#peres-wooters-game-ref2">[2]</a></sup>. In the game, a player A thinks of a number (0,1 or 2) and the opponent, player B, tries to guess any number but the one chosen by player A. 
>
> Classically, if you just made a guess, you'd have to ask two questions to be right $100\%$ of the time. If instead, player A prepares a qubit with 0, 1, or 2 encoded into three single qubit states that are at an angle of 120 degrees with respect to each other and then hands the state to the opponent, then player B can apply a Positive Operator Valued Measure (POVM) consisting of 3 states that are perpendicular to the states chosen by player A. 
> It can be shown that this allows B to be right $100\%$ of the time with only 1 measurement, which is something that is not achievable with a von Neumann measurement on 1 qubit.
See also Peres<sup><a href="#peres-wooters-game-ref3">[3, chapter 9.6]</a></sup> for a nice description of the optimal POVM.

Next, we address how we can implement the mentioned POVM by way of a von Neumann measurement, and then how to implement said von Neumann measurement in Q\#. First, we note that the POVM elements are given by the columns of the following matrix: 
 
$$M = \frac{1}{\sqrt{2}}\left(\begin{array}{rrr}
1 & 1 & 1 \\ 
1 & \omega & \omega^2 
\end{array}
\right)$$

where $\omega = e^{2 \pi i/3}$ denotes a primitive $3$rd root of unity. Our task will be to implement the rank 1 POVM given by the columns of $M$ via a von Neumann measurement. This can be done by "embedding" $M$ into a larger unitary matrix (taking complex conjugates and transposed):

$$M' = \frac{1}{\sqrt{2}}\left(\begin{array}{cccc}
1 & -1 & 1 & 0 \\ 
1 & -\omega^2 & \omega & 0 \\
1 & -\omega & \omega^2 & 0 \\
0 & 0 & 0 & -i 
\end{array}
\right)$$

Notice that applying $M'$ to input states given by column $i$ of $M$ (padded with two zeros to make it a vector of length $4$), where $i=0, 1, 2$ will never return the label $i$ as the corresponding vectors are perpendicular. 

We are therefore left with the problem of implementing $M'$ as a sequence of elementary quantum gates. Notice that 

$$M' \cdot {\rm diag}(1,-1,1,-1) = M' \cdot (\mathbf{1}_2 \otimes Z) = 
\frac{1}{\sqrt{2}}\left(\begin{array}{cccc}
1 & 1 & 1 & 0 \\ 
1 & \omega^2 & \omega & 0 \\
1 & \omega & \omega^2 & 0 \\
0 & 0 & 0 & i 
\end{array}
\right)$$

Using a technique used in the Rader (also sometimes called Rader-Winograd) decomposition of the discrete Fourier transform <sup><a href="#peres-wooters-game-ref4">[4]</a></sup>, which reduces it to a cyclic convolution, we apply a $2\times 2$ Fourier transform on the indices $i,j=1,2$ of this matrix (i.e. a block matrix which consists of a direct sum of blocks $\mathbf{1}_1$, $H$, and $\mathbf{1}_1$ which we abbreviate in short as ${\rm diag}(1,H,1)$). This yields

$${\rm diag}(1, H, 1) \cdot M' \cdot (\mathbf{1}_2 \otimes Z) \cdot {\rm diag}(1, H, 1) = 
\left(\begin{array}{rrrr}
\frac{1}{\sqrt3} & \sqrt{\frac23} & 0 & 0 \\ 
\sqrt{\frac23} & -\frac{1}{\sqrt3} & 0 & 0 \\
0 & 0 & i & 0 \\
0 & 0 & 0 & i 
\end{array}
\right)$$

This implies that after multiplication with the diagonal operator $(S^\dagger \otimes \mathbf{1}_2)$, we are left with 

$${\rm diag}(1, H, 1) \cdot M' \cdot (\mathbf{1}_2 \otimes Z) \cdot {\rm diag}(1, H, 1)\cdot (S^\dagger \otimes \mathbf{1}_2) = 
\left(\begin{array}{rrrr}
\frac{1}{\sqrt3} & \sqrt{\frac23} & 0 & 0 \\ 
\sqrt{\frac23} & -\frac{1}{\sqrt3} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 
\end{array}
\right)$$

which is a (zero-)controlled rotation $R$ around the $Y$-axis by an angle given by $\arccos \sqrt{\frac23}$. Putting everything together, i.e., by applying the inverses of gates so that we find that 

$$M' = {\rm diag}(1,H,1) \cdot R \cdot (S \otimes \mathbf{1}_2) \cdot {\rm diag}(1,H,1) \cdot (\mathbf{1}_2 \otimes Z)$$

 Noting finally, that to apply this sequence of unitaries to a column vector, we have to apply it in reverse when writing it as a program (as actions on vectors are left-associative). After applying a circuit identity that simplifies the resulting circuit after implementing ${\rm diag}(1,H,1)$ via controlled $H$, 
we arrive at the listing shown below.

<a name="peres-wooters-game-ref1"></a>[1]  A. Holevo.  Information-theoretical aspects of quantum measurement.  Problems of Information Transmission, vol. 9, no. 2,pp. 110–118 (1973)  
<a name="peres-wooters-game-ref2"></a>[2]  A. Peres and W. K. Wootters.  Optimal detection of quantum information.  Phys. Rev. Lett., vol. 66, pp. 1119-1122, Mar.1991.  
<a name="peres-wooters-game-ref3"></a>[3]  A. Peres.  Quantum Theory:  Concepts and Methods.  Kluwer Academic Publishers, 2002.  
<a name="peres-wooters-game-ref4"></a>[4]  C. M. Rader.  Discrete Fourier transforms when the number of data samples is prime.  Proc. IEEE 56, 1107–1108 (1968

In [ ]:
%kata T203_IsQubitNotInABC_Test

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation IsQubitNotInABC (q : Qubit) : Int {
        
    let alpha = ArcCos(Sqrt(2.0 / 3.0));

    using (a = Qubit()) {
        Z(q);
        CNOT(a, q);
        Controlled H([q], a);
        S(a);
        X(q);

        (ControlledOnInt(0, Ry))([a], (-2.0 * alpha, q));
        CNOT(a, q);
        Controlled H([q], a);
        CNOT(a, q);

        // finally, measure in the standard basis
        let res0 = MResetZ(a);
        let res1 = M(q);

        // dispatch on the cases
        if (res0 == Zero and res1 == Zero) {
            return 0;
        }
        elif (res0 == One and res1 == Zero) {
            return 1;
        }
        elif (res0 == Zero and res1 == One) {
            return 2;
        }
        else {
            // this should never occur
            return 3;
        }
    }
}

[Return to task 2.3 of the Measurements kata.](./Measurements.ipynb#peres-wooters-game)